# Toki - Data expression library

`Toki` aims to be a data expression library written in `Rust`. Its main objetive is to offer a simple and intuitive API to handle data expressions that can be evaluated for different backends.

In the context of this document, a data expression is a way to express a data in with graph that will be evaluate later by demand.

For example, in `Python`, there are some libraries that work with this concepts, such as `dask`, `ibis-framework`, `sqlalchemy` and `metadsl`.

To ilustrate this concept, mathematic expressions are very useful:

```python
x = 0
y = x + 1
```

Using a common programing language, these lines are treated as statements and are evaluated automatically. But, if these lines were 
treated as expressions, the `y` value is unknown until the user calls the evaluation for `y` value.

Consider the follow example using `dask`:

```python
>>> import dask.array as da
>>> x = da.arange(10, chunks=2).sum()
>>> y = da.arange(10, chunks=2).mean()
>>> x2, y2 = optimize(x, y)

>>> x2.compute() == a.compute()
True
>>> y2.compute() == b.compute()
True
```

As it can be observed, at running time, x2 and y2 values are unknown until user calls the `compute` method.

At this moment, there are some similar data expression libraries written in `Rust`, such as [Diesel](https://docs.diesel.rs/), etc.

Consider the follow code using `Diesel`:

```rust
let data = animals
    .select(species)
    .filter(name.is_null())
    .first::<String>(&connection)?;
```

The `Toki`'s goal is to allow the same operation but using a simpler approach:

```rust
let data = animals[animals[species].name.is_null()].head(1);
```

This document explores `Rust` in a way to achive this goal.

## Data Expression Code Design

Some common elements that a data expression can have:

- Data Type expression (literal types, such as Integer32, String)
- Table expression (such as table, columns, etc)
- Operation expression

## Rust language structure

Compared with other languages, Rust can be quite challenging. Instead of classes, it should be constructed using structs and traits.
The follow code tries to create an initial proposal for `Toki`:

In [2]:
use std::fmt;
use std::ops;

In [3]:
trait Expression {
    fn __str__(&self) -> &str;
}

trait DataType {}

trait NumericType {}

impl Expression for dyn DataType {
    fn __str__(&self) -> &str {
        &"DataType"
    }
}

impl Expression for dyn NumericType {
    fn __str__(&self) -> &str {
        &"NumericType"
    }
}

impl fmt::Debug for dyn Expression {
    fn fmt(&self, f: &mut fmt::Formatter<'_>) -> fmt::Result {
        f.debug_struct(self.__str__()).finish()
    }
}

impl fmt::Display for dyn Expression {
    fn fmt(&self, formatter: &mut fmt::Formatter) -> fmt::Result {
        formatter.write_str(self.__str__())
    }
}


#[derive(Debug)]
enum NumberByteSize {
    Integer32 = 4,
    Integer64 = 8,
}


#[derive(Debug)]
struct Integer32 {
    parent: Option<Box<dyn Expression>>,
    value: i32,
}


impl Expression for Integer32 {
    fn __str__(&self) -> &str {
        "Integer32"
    }
}

impl DataType for Integer32 {}
impl NumericType for Integer32 {}

impl Integer32 {
    fn new(value: i32) -> Integer32 {
        Integer32 { value: value , parent: None }
    }
    
    // fn new_with_parent(value: i32, parent: Option<dyn Expression + 'static>) -> Integer32 {
    //     Integer32 { value: value , parent: Some<parent>}
    // }
}


#[derive(Debug)]
struct Integer64 {
    parent: Option<Box<dyn Expression>>,
    value: i64,
}


impl Integer64 {
    fn new(value: i64) -> Integer64 {
        Integer64 { value: value , parent: None }
    }
    
    // fn new_with_parent(value: i32, parent: Option<dyn Expression + 'static>) -> Integer32 {
    //     Integer32 { value: value , parent: Some<parent>}
    // }
}

impl Expression for Integer64 {
    fn __str__(&self) -> &str {
        "Integer64"
    }
}

impl DataType for Integer64 {}
impl NumericType for Integer64 {}


let obj_i32: Integer32 = Integer32::new(1);
println!("{:?}", obj_i32);

let obj_i64 = Integer64::new(2);
println!("{:?}", obj_i64);


// OPERATION

trait BinaryOp {
    fn resolve_expression();
}

#[derive(Debug)]
struct Add {
    left: Box<dyn Expression>,
    right: Box<dyn Expression>,
}

// impl BynaryOp for Add {}


impl ops::Add<Integer32> for Integer32 {
    type Output = Add;

    fn add(self, rhs: Integer32) -> Add {
        Add {
            left: Box::new(self),
            right: Box::new(rhs)
        }
    }
}

impl ops::Add<Integer64> for Integer64 {
    type Output = Add;

    fn add(self, rhs: Integer64) -> Add {
        Add {
            left: Box::new(self),
            right: Box::new(rhs)
        }
    }
}

impl ops::Add<Integer64> for Integer32 {
    type Output = Add;

    fn add(self, rhs: Integer64) -> Add {
        Add {
            left: Box::new(self),
            right: Box::new(rhs)
        }
    }
}

impl ops::Add<Integer32> for Integer64 {
    type Output = Add;

    fn add(self, rhs: Integer32) -> Add {
        Add {
            left: Box::new(self),
            right: Box::new(rhs)
        }
    }
}


// #[derive(Debug)]
// struct SchemaField {
//     name: String,
//     nullable: bool,
//     extra: String,
// }


// #[derive(Debug)]
// struct Schema {
//     name: String,
// }


// #[derive(Debug)]
// struct TableExpr<T> {
//     parent: Option<T>,
//     name: String,
//     schema: Schema,
// }


let x: Integer32 = Integer32::new(1);
let y: Integer32 = Integer32::new(2);
println!("{:?}", x + y);

let x = Integer64::new(1);
let y = Integer64::new(2);
println!("{:?}", x + y);


let x = Integer32::new(1);
let y = Integer64::new(2);
println!("{:?}", x + y);

let x = Integer64::new(1);
let y = Integer32::new(2);
println!("{:?}", x + y);

Integer32 { parent: None, value: 1 }
Integer64 { parent: None, value: 2 }
Add { left: Integer32, right: Integer32 }
Add { left: Integer64, right: Integer64 }
Add { left: Integer32, right: Integer64 }
Add { left: Integer64, right: Integer32 }


## References

- Rust
  - https://doc.rust-lang.org/std/fmt/trait.Debug.html